<a href="https://colab.research.google.com/github/fralfaro/r4ds-book/blob/main/docs/14_strings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strings

## Introduction

So far, you've used a bunch of strings without learning much about the details.
Now it's time to dive into them, learning what makes strings tick, and mastering some of the powerful string manipulation tool you have at your disposal.

We'll begin with the details of creating strings and character vectors.
You'll then dive into creating strings from data.
Next, we'll discuss the basics of regular expressions, a powerful tool for describing patterns in strings, then use those tools to extract data from strings.
The chapter finishes up with functions that work with individual letters, a brief discussion of where your expectations from English might steer you wrong when working with other languages, and a few useful non-stringr functions.

This chapter is paired with two other chapters.
Regular expression are a big topic, so we'll come back to them again in [Chapter -@sec-regular-expressions]. We'll also come back to strings again in [Chapter -@sec-programming-with-strings] where we'll look at them from a programming perspective rather than a data analysis perspective.

### Prerequisites

In this chapter, we'll use functions from the stringr package which is part of the core tidyverse.
We'll also use the babynames data since it provides some fun strings to manipulate.




```{r}
#| label: setup
#| message: false

library(tidyverse)
library(babynames)
```



Similar functionality is available in base R (through functions like `grepl()`, `gsub()`, and `regmatches()`) but we think you'll find stringr easier to use because it's been carefully designed to be as consistent as possible.
You can easily tell when you're using a stringr function because all stringr functions start with `str_`.
This is particularly useful if you use RStudio, because typing `str_` will trigger autocomplete, allowing you jog your memory of which functions are available.




```{r}
#| echo: false

knitr::include_graphics("screenshots/stringr-autocomplete.png")
```



## Creating a string

We've created strings in passing earlier in the book, but didn't discuss the details.
First, you can create a string using either single quotes (`'`) or double quotes (`"`).
Unlike other languages, there is no difference in behavior, but in the interests of consistency the [tidyverse style guide](https://style.tidyverse.org/syntax.html#character-vectors) recommends using `"`, unless the string contains multiple `"`.



```{r}
string1 <- "This is a string"
string2 <- 'If I want to include a "quote" inside a string, I use single quotes'
```



If you forget to close a quote, you'll see `+`, the continuation character:

    > "This is a string without a closing quote
    + 
    + 
    + HELP I'M STUCK

If this happen to you and you can't figure out which quote you need to close, press Escape to cancel, and try again.

### Escapes

To include a literal single or double quote in a string you can use `\` to "escape" it:



```{r}
double_quote <- "\"" # or '"'
single_quote <- '\'' # or "'"
```



And if you want to include a literal backslash in your string, you'll need to double it up: `"\\"`:

```{r}
backslash <- "\\"
```



Beware that the printed representation of a string is not the same as string itself, because the printed representation shows the escapes (in other words, when you print a string, you can copy and paste the output to recreate that string).
To see the raw contents of the string, use `str_view()`[^strings-1]:

[^strings-1]: You can also use the base R function `writeLines()`




```{r}
x <- c(single_quote, double_quote, backslash)
x
str_view(x)
```




### Raw strings {#sec-raw-strings}

Creating a string with multiple quotes or backslashes gets confusing quickly.
To illustrate the problem, lets create a string that contains the contents of the chunk where I define the `double_quote` and `single_quote` variables:



```{r}
tricky <- "double_quote <- \"\\\"\" # or '\"'
single_quote <- '\\'' # or \"'\""
str_view(tricky)
```



That's a lot of backslashes!
(This is sometimes called [leaning toothpick syndome](https://en.wikipedia.org/wiki/Leaning_toothpick_syndrome).) To eliminate the escaping you can instead use a **raw string**[^strings-2]:

[^strings-2]: Available in R 4.0.0 and above.



```{r}
tricky <- r"(double_quote <- "\"" # or '"'
single_quote <- '\'' # or "'")"
str_view(tricky)
```



A raw string usually starts with `r"(` and finishes with `)"`.
But if your string contains `)"` you can instead use `r"[]"` or `r"{}"`, and if that's still not enough, you can insert any number of dashes to make the opening and closing pairs unique, e.g. `` `r"--()--" ``, `` `r"---()---" ``, etc. Raw strings are flexible enough to handle any text.

### Other special characters

As well as `\"`, `\'`, and `\\` there are a handful of other special characters that may come in handy. The most common are `\n`, newline, and `\t`, tab. You'll also sometimes see strings containing Unicode escapes that start with `\u` or `\U`. This is a way of writing non-English characters that works on all systems. You can see the complete list of other special characters in `?'"'`.



```{r}
x <- c("one\ntwo", "one\ttwo", "\u00b5", "\U0001f604")
x
str_view(x)
```



Note that `str_view()` shows special whitespace characters (i.e. everything except spaces and newlines) with a blue background to make them easier to spot.

### Vectors {#sec-string-vector}

You can combine multiple strings into a character vector by using `c()`:



```{r}
x <- c("first string", "second string", "third string")
x
```



Technically, a string is a length-1 character vector, but this doesn't have much bearing on your data analysis life.
We'll come back to this idea is more detail when we think about vectors as a programming tool in [Chapter -@sec-vectors].

### Exercises

## Creating strings from data

Now that you've learned the basics of creating strings by "hand", we'll go into the details of creating strings from other strings.
It's a common problem: you often have some fixed strings that you wrote that you want to combine some varying strings that come from the data.
For example, to create a greeting you might combine "Hello" with a `name` variable.
First, we'll discuss two functions that make this easy.
Then we'll talk about a slightly different scenario where you want to summarise a character vector, collapsing any number of strings into one.

### `str_c()`

`str_c()`[^strings-3] takes any number of vectors as arguments and returns a character vector:

[^strings-3]: `str_c()` is very similar to the base `paste0()`.
    There are two main reasons I recommend: it obeys the usual rules for handling `NA` and it uses the tidyverse recycling rules.




```{r}
str_c("x", "y")
str_c("x", "y", "z")
str_c("Hello ", c("John", "Susan"))
```




`str_c()` is designed to be used with `mutate()` so it obeys the usual rules for recycling and missing values:



```{r}
df <- tibble(name = c("Timothy", "Dewey", "Mable", NA))
df |> mutate(greeting = str_c("Hi ", name, "!"))
```



If you want missing values to display in some other way, use `coalesce()` either inside or outside of `str_c()`:



```{r}
df |> mutate(
  greeting1 = str_c("Hi ", coalesce(name, "you"), "!"),
  greeting2 = coalesce(str_c("Hi ", name, "!"), "Hi!")
)
```



### `str_glue()`

If you are mixing many fixed and variable strings with `str_c()`, you'll notice that you have to type `""` repeatedly, and this can make it hard to see the overall goal of the code.
An alternative approach is provided by the [glue package](https://glue.tidyverse.org) via `str_glue()`[^strings-4] .
You give it a single string containing `{}` and anything inside `{}` will be evaluated like it's outside of the string:

[^strings-4]: If you're not using stringr, you can also access it directly with `glue::glue().`



```{r}
df |> mutate(greeting = str_glue("Hi {name}!"))
```



You can use any valid R code inside of `{}`, but it's a good idea to pull complex calculations out into their own variables so you can more easily check your work.

As you can see above, `str_glue()` currently converts missing values to the string "NA" making it slightly inconsistent with `str_c()`.
We'll hopefully fix that by the time the book is printed: <https://github.com/tidyverse/glue/issues/246>

You also might wonder what happens if you need to include a regular `{` or `}` in your string.
You might expect that you'll need to escape it, and you'd be right.
But glue uses a slightly different escaping technique; instead of prefixing with special character like `\`, you just double up the `{` and `}`:



```{r}
df |> mutate(greeting = str_glue("{{Hi {name}!}}"))
```



### `str_flatten()`

`str_c()` and `glue()` work well with `mutate()` because their output is the same length as their inputs.
What if you want a function that works well with `summarise()`, i.e. something that always returns a single string?
That's the job of `str_flatten()`[^strings-5]: it takes a character vector and combines each element of the vector into a single string:

[^strings-5]: The base R equivalent is `paste()` used with the `collapse` argument.



```{r}
str_flatten(c("x", "y", "z"))
str_flatten(c("x", "y", "z"), ", ")
str_flatten(c("x", "y", "z"), ", ", last = ", and ")
```



This makes it work well with `summarise()`:

```{r}
df <- tribble(
  ~ name, ~ fruit,
  "Carmen", "banana",
  "Carmen", "apple",
  "Marvin", "nectarine",
  "Terence", "cantaloupe",
  "Terence", "papaya",
  "Terence", "madarine"
)
df |>
  group_by(name) |> 
  summarise(fruits = str_flatten(fruit, ", "))
```



### Exercises

1.  Compare and contrast the results of `paste0()` with `str_c()` for the following inputs:



    ```{r}
    #| eval: false

    str_c("hi ", NA)
    str_c(letters[1:2], letters[1:3])
    ```



2.  Convert the following expressions from `str_c()` to `glue()` or vice versa:

    a.  `str_c("The price of ", food, " is ", price)`

    b.  `glue("I'm {age} years old and live in {country}")`

    c.  `str_c("\\section{", title, "}")`

## Working with patterns

It's probably even more useful to be able to extract data from string than create strings from data, but before we can tackle that, we need to take a brief digression to talk about **regular expressions**.
Regular expressions are a very concise language that describes patterns in strings.
For example, `"^The"` is shorthand for any string that starts with "The", and `a.+e` is a shorthand for "a" followed by one or more other characters, followed by an "e".

We'll start by using `str_detect()` which answers a simple question: "does this pattern occur anywhere in my vector?".
We'll then ask progressively more complex questions by learning more about regular expressions and the stringr functions that use them.

### Detect matches

The term "regular expression" is a bit of a mouthful, so most people abbreviate to "regex"[^strings-6] or "regexp".
To learn about regexes, we'll start with the simplest function that uses them: `str_detect()`. It takes a character vector and a pattern, and returns a logical vector that says if the pattern was found at each element of the vector.
The following code shows the simplest type of pattern, an exact match.

[^strings-6]: With a hard g, sounding like "reg-x".



```{r}
x <- c("apple", "banana", "pear")
str_detect(x, "e")   # does the word contain an e?
str_detect(x, "b")   # does the word contain a b?
str_detect(x, "ear") # does the word contain "ear"?
```



`str_detect()` returns a logical vector the same length as the first argument, so it pairs well with `filter()`.
For example, this code finds all names that contain a lower-case "x":




```{r}
babynames |> filter(str_detect(name, "x"))
```



We can also use `str_detect()` with `summarize()` by remembering that when you use a logical vector in a numeric context, `FALSE` becomes 0 and `TRUE` becomes 1.
That means `sum(str_detect(x, pattern))` will tell you the number of observations that match, while `mean(str_detect(x, pattern))` tells you the proportion of observations that match.
For example, the following snippet computes and visualizes the proportion of baby names that contain "x", broken down by year:




```{r}
#| fig-alt: >
#|   A timeseries showing the proportion of baby names that contain the letter x. 
#|   The proportion declines gradually from 8 per 1000 in 1880 to 4 per 1000 in 
#|   1980, then increases rapidly to 16 per 1000 in 2019.

babynames |> 
  group_by(year) |> 
  summarise(prop_x = mean(str_detect(name, "x"))) |> 
  ggplot(aes(year, prop_x)) + 
  geom_line()
```



(Note that this gives us the proportion of names that contain an x; if you wanted the proportion of babies given a name containing an x, you'd need to perform a weighted mean).

### Introduction to regular expressions

The simplest patterns, like those above, are exact: they match any strings that contain the exact sequence of characters in the pattern:




```{r}
str_detect(c("x", "X"), "x")
str_detect(c("xyz", "xza"), "xy")
```

In general, any letter or number will match exactly, but punctuation characters like `.`, `+`, `*`, `[`, `]`, `?`, often have special meanings[^strings-7].
For example, `.`
will match any character[^strings-8], so `"a."` will match any string that contains an "a" followed by another character
:

[^strings-7]: You'll learn how to escape this special behaviour in @sec-regexp-escaping

[^strings-8]: Well, any character apart from `\n`.

```{r}
str_detect(c("a", "ab", "ae", "bd", "ea", "eab"), "a.")
```




To get a better sense of what's happening, I'm going to switch to `str_view_all()`.
This shows which characters are matched by surrounding it with `<>` and coloring it blue:




```{r}
str_view_all(c("a", "ab", "ae", "bd", "ea", "eab"), "a.")
```



Regular expressions are a powerful and flexible language which we'll come back to in [Chapter -@sec-regular-expressions].
Here I'll just introduce only the most important components: quantifiers and character classes.

**Quantifiers** control how many times an element that can be applied to other pattern: `?` makes a pattern optional (i.e. it matches 0 or 1 times), `+` lets a pattern repeat (i.e. it matches at least once), and `*` lets a pattern be optional or repeat (i.e. it matches any number of times, including 0).




```{r}
# ab? matches an "a", optionally followed by a "b".
str_view_all(c("a", "ab", "abb"), "ab?")

# ab+ matches an "a", followed by at least one "b".
str_view_all(c("a", "ab", "abb"), "ab+")

# ab* matches an "a", followed by any number of "b"s.
str_view_all(c("a", "ab", "abb"), "ab*")
```



**Character classes** are defined by `[]` and let you match a set set of characters, e.g. `[abcd]` matches "a", "b", "c", or "d".
You can also invert the match by starting with `^`: `[^abcd]` matches anything **except** "a", "b", "c", or "d".
We can use this idea to find the vowels in a few particularly special names:




```{r}
names <- c("Hadley", "Mine", "Garrett")
str_view_all(names, "[aeiou]")
```



You can combine character classes and quantifiers.
Notice the difference between the following two patterns that look for consonants.
The same characters are matched, but the number of matches is different.




```{r}
str_view_all(names, "[^aeiou]")
str_view_all(names, "[^aeiou]+")
```



Regular expressions are very compact and use a lot of punctuation characters, so they can seem overwhelming at first, and you'll think a cat has walked across your keyboard.
So don't worry if they're hard to understand at first; you'll get better with practice.
Lets start that practice with some other useful stringr functions.

### Count matches

A variation on `str_detect()` is `str_count()`: rather than a simple yes or no, it tells you how many matches there are in a string:




```{r}
x <- c("apple", "banana", "pear")
str_count(x, "p")
```



Note that regular expression matches never overlap so `str_count()` only starts looking for a new match after the end of the last match.
For example, in `"abababa"`, how many times will the pattern `"aba"` match?
Regular expressions say two, not three:




```{r}
str_count("abababa", "aba")
str_view_all("abababa", "aba")
```




It's natural to use `str_count()` with `mutate()`.
The following example uses `str_count()` with character classes to count the number of vowels and consonants in each name.



```{r}
babynames |> 
  count(name) |> 
  mutate(
    vowels = str_count(name, "[aeiou]"),
    consonants = str_count(name, "[^aeiou]")
  )
```



If you look closely, you'll notice that there's something off with our calculations: "Aaban" contains three "a"s, but our summary reports only two vowels.
That's because I've forgotten to tell you that regular expressions are case sensitive.
There are three ways we could fix this:

-   Add the upper case vowels to the character class: `str_count(name, "[aeiouAEIOU]")`.
-   Tell the regular expression to ignore case: `str_count(regex(name, ignore.case = TRUE), "[aeiou]")`. We'll talk about this next.
-   Use `str_to_lower()` to convert the names to lower case: `str_count(str_to_lower(name), "[aeiou]")`. We'll come back to this function in @sec-other-languages.

This is pretty typical when working with strings --- there are often multiple ways to reach your goal, either making your pattern more complicated or by doing some preprocessing on your string.
If you get stuck trying one approach, it can often be useful to switch gears and tackle the problem from a different perspective.

### Replace matches

`str_replace_all()` allows you to replace a match with the text of your choosing.
This can be particularly useful if you need to standardize a vector.
Unlike the regexp functions we've encountered so far, `str_replace_all()` takes three arguments: a character vector, a pattern, and a replacement.

The simplest use is to replace a pattern with a fixed string:



```{r}
x <- c("apple", "pear", "banana")
str_replace_all(x, "[aeiou]", "-")
```



`str_remove_all()` is a short cut for `str_replace_all(x, pattern, "")` --- it removes matching patterns from a string.

Use in `mutate()`

Using pipe inside mutate.
Recommendation to make a function, and think about testing it --- don't need formal tests, but useful to build up a set of positive and negative test cases as you.

### Advanced replacements

You can also perform multiple replacements by supplying a named vector.
The name gives a regular expression to match, and the value gives the replacement.



```{r}
x <- c("1 house", "1 person has 2 cars", "3 people")
str_replace_all(x, c("1" = "one", "2" = "two", "3" = "three"))
```



Alternatively, you can provide a replacement function: it's called with a vector of matches, and should return what to replacement them with.
We'll come back to this powerful tool in [Chapter -@sec-programming-with-strings].




```{r}
x <- c("1 house", "1 person has 2 cars", "3 people")
str_replace_all(x, "[aeiou]+", str_to_upper)
```



### Pattern control

Now that you've learn about regular expressions, you might be worried about them working when you don't want them to.
You can opt-out of the regular expression rules by using `fixed()`:




```{r}
str_view(c("", "a", "."), fixed("."))
```



Both fixed strings and regular expressions are case sensitive by default.
You can opt out by setting `ignore_case = TRUE`.



```{r}
str_view_all("x  X  xy", "X")
str_view_all("x  X  xy", fixed("X", ignore_case = TRUE))
str_view_all("x  X  xy", regex(".Y", ignore_case = TRUE))
```



### Exercises

1.  What name has the most vowels?
    What name has the highest proportion of vowels?
    (Hint: what is the denominator?)

2.  For each of the following challenges, try solving it by using both a single regular expression, and a combination of multiple `str_detect()` calls.

    a.  Find all words that start or end with `x`.
    b.  Find all words that start with a vowel and end with a consonant.
    c.  Are there any words that contain at least one of each different vowel?

3.  Replace all forward slashes in a string with backslashes.

4.  Implement a simple version of `str_to_lower()` using `str_replace_all()`.

5.  Switch the first and last letters in `words`.
    Which of those strings are still `words`?

## Extract data from strings

Common for multiple variables worth of data to be stored in a single string.
In this section you'll learn how to use various functions tidyr to extract them.

Waiting on: <https://github.com/tidyverse/tidyups/pull/15>

## Locale dependent operations {#sec-other-languages}

So far all of our examples have been using English.
The details of the many ways other languages are different to English are too diverse to detail here, but I wanted to give a quick outline of the functions who's behavior differs based on your **locale**, the set of settings that vary from country to country.

Locale is specified with lower-case language abbreviation, optionally followed by a `_` and a upper-case region identifier.
For example, "en" is English, "en_GB" is British English, and "en_US" is American English.
If you don't already know the code for your language, [Wikipedia](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes) has a good list, and you can see which are supported with `stringi::stri_locale_list()`.

Base R string functions automatically use your locale current locale.
This means that string manipulation code works the way you expect when you're working with text in your native language, but it might work differently when you share it with someone who lives in another country.
To avoid this problem, stringr defaults to the "en" locale, and requires you to specify the `locale` argument to override it.
This also makes it easy to tell if a function might have different behavior in different locales.

Fortunately there are three sets of functions where the locale matters:

-   **Changing case**: while only relatively few languages have upper and lower case (Latin, Greek, and Cyrillic, plus a handful of lessor known languages).
    The rules are not te same in every language that uses these alphabets.
    For example, Turkish has two i's: with and without a dot, and it has a different rule for capitalising them:



    ```{r}
    str_to_upper(c("i", "ı"))
    str_to_upper(c("i", "ı"), locale = "tr")
    ```



-   **Comparing strings**: `str_equal()` lets you compare if two strings are equal, optionally ignoring case:



    ```{r}
    str_equal("i", "I", ignore_case = TRUE)
    str_equal("i", "I", ignore_case = TRUE, locale = "tr")
    ```



-   **Sorting strings**: `str_sort()` and `str_order()` sort vectors alphabetically, but the alphabet is not the same in every language[^strings-9]!
    Here's an example: in Czech, "ch" is a compound letter that appears after `h` in the alphabet.



    ```{r}
    str_sort(c("a", "c", "ch", "h", "z"))
    str_sort(c("a", "c", "ch", "h", "z"), locale = "cs")
    ```



    Danish has a similar problem.
    Normally, characters with diacritics (e.g. à, á, â) sort after the plain character (e.g. a).
    But in Danish ø and å are their own letters that come at the end of the alphabet:



    ```{r}
    str_sort(c("a", "å", "o", "ø", "z"))
    str_sort(c("a", "å", "o", "ø", "z"), locale = "da")
    ```



  This also comes up when sorting strings with `dplyr::arrange()` which is why it also has a `locale` argument.

[^strings-9]: Sorting in languages that don't have an alphabet (like Chinese) is more complicated still.

## Letters

Functions that work with the components of strings called **code points**.
Depending on the language involved, this might be a letter (like in most European languages), a syllable (like Japanese), or a logogram (like in Chinese).
It might be something more exotic like an accent, or a special symbol used to join two emoji together.
But to keep things simple, I'll call these letters.

### Length

`str_length()` tells you the number of letters in the string:



```{r}
str_length(c("a", "R for data science", NA))
```



You could use this with `count()` to find the distribution of lengths of US babynames, and then with `filter()` to look at the longest names[^strings-10]:

[^strings-10]: Looking at these entries, I'd say the babynames data removes spaces or hyphens from names and truncates after 15 letters.



```{r}
babynames |>
  count(length = str_length(name), wt = n)

babynames |> 
  filter(str_length(name) == 15) |> 
  count(name, wt = n, sort = TRUE)
```



### Subsetting

You can extract parts of a string using `str_sub(string, start, end)`.
The `start` and `end` arguments are inclusive, so the length of the returned string will be `end - start + 1`:



```{r}
x <- c("Apple", "Banana", "Pear")
str_sub(x, 1, 3)
```



You can use negative values to count back from the end of the string: -1 is the last character, -2 is the second to last character, etc.



```{r}
str_sub(x, -3, -1)
```



Note that `str_sub()` won't fail if the string is too short: it will just return as much as possible:



```{r}
str_sub("a", 1, 5)
```



We could use `str_sub()` with `mutate()` to find the first and last letter of each name:




```{r}
babynames |> 
  mutate(
    first = str_sub(name, 1, 1),
    last = str_sub(name, -1, -1)
  )
```



### Long strings

Sometimes the reason you care about the length of a string is because you're trying to fit it into a label on a plot or in a table.
stringr provides two useful tools for cases where your string is too long:

-   `str_trunc(x, 20)` ensures that no string is longer than 20 characters, replacing any thing too long with `…`.

-   `str_wrap(x, 20)` wraps a string introducing new lines so that each line is at most 20 characters (it doesn't hyphenate, however, so any word longer than 20 characters will make a longer time)



```{r}
x <- "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."

str_trunc(x, 30)
str_view(str_wrap(x, 30))
```



TODO: add example with a plot.

### Exercises

1.  Use `str_length()` and `str_sub()` to extract the middle letter from each baby name. What will you do if the string has an even number of characters?
2.  Are there any major trends in the length of babynames over time? What about the popularity of first and last letters?

## Other functions

The are a bunch of other places you can use regular expressions outside of stringr.

-   `matches()`: as you can tell from it's lack of `str_` prefix, this isn't a stringr fuction.
    It's a "tidyselect" function, a fucntion that you can use anywhere in the tidyverse when selecting variables (e.g. `dplyr::select()`, `rename_with()`, `across()`, ...).

-   `apropos()` searches all objects available from the global environment.
    This is useful if you can't quite remember the name of the function.



    ```{r}
    apropos("replace")
    ```



-   `dir()` lists all the files in a directory.
    The `pattern` argument takes a regular expression and only returns file names that match the pattern.
    For example, you can find all the R Markdown files in the current directory with:



    ```{r}
    head(dir(pattern = "\\.Rmd$"))
    ```



    (If you're more comfortable with "globs" like `*.Rmd`, you can convert them to regular expressions with `glob2rx()`).
